In [1]:
# In Class Lab 3
# G. Besla 

# import relevant modules 
import astropy.units as u
import numpy as np
from astropy import constants as const # import astropy constants

The Large Magellanic Cloud is at a distance of 50 kpc from the Galactic Center. 
It is observed to have a stellar disk that extends to a radius of at least 18.5 kpc.

![LMC](./Lab3_Tidal.png)
Deep photometric imaging reveals the faint stellar outskirts of the LMC. 
Outskirts: DECam data Mackey+2016 MNRAS 459, 239. 
Inner: shallower imaging from robotic telescopes Besla+2016 APJ 825.

In this lab we will determine
the minimum mass required for the LMC so that it maintains the observed radius 
in the face of the Milky Way's tidal field. 

# Part A

We define the mass profile of the Milky Way using a Hernquist profile.


$\rho(r) =  \frac{M_{halo}}{2\pi} \frac{a}{r(r+a)^3} \qquad M(r) =  \frac{M_{halo} r^2}{(a+r)^2}$ 



## #1

Create a function `hernquist_mass` that returns the dark matter halo mass at a given radius in units of solar mass.
This function should take as input:  the distance from the Galactic center $r$, the scale radius $a$, and the halo mass $M_{halo}$.


For the Hernquist scale radius for the Milky Way, use the default value of $a=60$ kpc. 

For $M_{halo}$ use your answer for the total mass of the simulated Milky Way you computed in Homework 3 as the default value (in units of 1e12). 

In [6]:
def hernquist_mass(r,a=60*u.kpc,MHalo=1.975*u.Msun):
    '''
    This function finds the dark matter halo mass for a given radius r based on the Hernquist profile
    
    
    Inputs
        r: astropy quantity
            This is the distance from the galactic center in units of kpc
        a: astropy quantity
            This is the scale radius for the Hernquist profile. The default value is 60 kpc for the Milky Way
        MHalo: astropy quantity
            The total dark matter halo mass in units of 10^12 Msun. Default value is that from HW3
    returns
        hern_mass: astropy quantity
            The dark matter halo mass at a given radius in units of MSun
    '''
    hern_mass = (MHalo*1e12) * r**2 / (a+r)**2
    return hern_mass
    

In [7]:
# compute mass within 10000 kpc
print(f"{hernquist_mass(10000*u.kpc):.2e}")

1.95e+12 solMass


In [8]:
# compute mass within 300 kpc
print(f"{hernquist_mass(300*u.kpc):.2e}")

1.37e+12 solMass


## #2

Compute the total mass of the Milky Way within 50 kpc, including its baryonic components (Dark Matter + Bulge + Disk). Use your answers from Homework 3 for the Bulge and Disk Masses. 
Store this as a variable called `mass_MW50`.


In [9]:
bulge_mass = .01*1e12 *u.Msun #bulge mass in MSun based on HW3
disk_mass = .075*1e12 *u.Msun #disk mass in MSun based on HW3
dm_mass= hernquist_mass(r=50*u.kpc) 
mass_MW50 = bulge_mass+disk_mass+dm_mass
print(f"{mass_MW50:.2e}")
    

4.93e+11 solMass


# Part B

The Jacobi Radius for a satellite on a circular orbit about an extended host, where 
the host is assumed to be well modeled as an isothermal sphere halo:


$R_j = r  \bigg( \frac{M_{sat}}{2 M_{host}(<r)} \bigg)^{1/3}$


The Isothermal Sphere approximation is not a bad one within 50 kpc.

Note also that the LMC is not on a circular orbit, but it is very close to its pericentric approach, where the velocity is all in the tangential component. So this isn't a terrible approximation either. 

## #1
Create a function called `jacobi_mass` that returns the total mass of a satellite galaxy in units of Msun, 
such that it has a given size 

Do this by rearranging the Jacobi Radius equation to solve for the satellite mass. 


In [10]:
def jacobi_mass(R_jacobi,r,MHost):
    '''
    This function determines the mass of a satellite based on the host 
    galaxy's mass and the satellite's orbital radius
    
    Inputs
        R_Jacobi: astropy quantity
            The radius of the satellite galaxy in kpc
        r: astropy quantity
            the distance in kpc between the host and satellite galaxy
        MHost: astropy quantity
            The mass of the host galaxy in units of Msun
            
    returns
        m_sat: astropy quantity
            minimum mass of the satellite galaxy
    '''
    M_sat= 2*MHost*(R_jacobi/r)**3
    return M_sat

## #2 

Determine the minimum total mass of the LMC needed to maintain its radius of 18.5 kpc in the face of the Milky Way's tidal 
field at its current distance of 50 kpc. Store this as a variable called `LMC_jacobiM`.

In [11]:
LMC_jacobiM = jacobi_mass(18.5*u.kpc,50*u.kpc,mass_MW50)
print(f'{LMC_jacobiM:.2e}')

4.99e+10 solMass


## #3

Recall that, ignoring centrifugal forces and assuming the host is a point mass, the tidal radius is given as :

$r_{tide} = r\left (\frac{m_{sat}}{4M_{host} } \right)^{1/3} $

Create a function to determine the total mass the must LMC possess to have a disk with radius 18.5 kpc.

The function should take as input, the current size of the satellite (kpc), this distance to the host (kpc) and the mass of the host (in Msun)

Use the function to determine the needed LMC mass and store it as a variable called `LMC_tidalM`. 

In [12]:
def jacobi_tidal(r_tide,r,MHost):
    '''
    This function determines the minimum mass needed to maintain a satellite based on the host 
    galaxy's mass and the satellite's tidal radius
    
    Inputs
        R_tide: astropy quantity
            The tidal radius of the satellite galaxy in kpc
        r: astropy quantity
            the distance in kpc between the host and satellite galaxy
        MHost: astropy quantity
            The mass of the host galaxy in units of Msun
    '''
    M_min= 4*MHost*(r_tide/r)**3
    return M_min

LMC_tidalM= jacobi_tidal(18.5*u.kpc,50*u.kpc,mass_MW50)
print(f'{LMC_tidalM:.2e}')

9.99e+10 solMass


## #4

a) Compare `LMC_tidalM` to the calculation using the Jacobi Radius.

b)How does the total mass of the LMC compare to its stellar mass (M$_\ast = 3 \times 10^9$ M$_\odot$)? 


In [13]:
print(LMC_tidalM/LMC_jacobiM) #tidalM is double the mass based on the jacobi radius

2.0


In [14]:
print(LMC_jacobiM/(3e9*u.Msun)) # total mass is roughly 16.6 times higher than the stellar mass

16.649906225895318


# Part C: Consistency Check


The dynamical mass of the LMC at any radius can be determined by assuming a flat rotation curve.  "Dynamical mass" means mass needed to explain the rotation curve. 

$V_c^2 = \frac{G M}{r} = constant$
 
 The rotation curve of the LMC is observed to flatten at a value of 91.7 +/- 18.8 km/s  (van der Marel & Kallivayalil 2014 ApJ 781)

  
   Create a function called `dyn_mass` that takes as input Vc (km/s) and distance to from the center of the galaxy (r) and returns the maximal dynamical mass in Msun. 
 
$G$ = 4.4985e-6 kpc$^3$/Gyr$^2$/M$_\odot$

  
## #1  
Compute the dynamical mass enclosed by the LMC within the observed radius. Store it as a variable called `LMC_dynM`. 

## #2

Is `LMC_dynM` consistent with `LMC_jacobiM`, the minimum mass needed to explain the observed radius of the LMC given the tidal field of the MW? If not, how can the numbers be reconciled?